# Enabling licensed solvers on Deepnote
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).


In [ ]:
import os
assert "DEEPNOTE_PROJECT_ID" in os.environ, "This tutorial is meant to be run on Deepnote"

Drake's MathematicalProgram interface supports some commercial solvers that require users to provide a license file in order to activate them.  This tutorial provides an example of how this can be accomplished on Deepnote; attempting to simplify the workflow but also minimize the chances of you accidentally sharing your license file.

You can **duplicate this notebook**, and upload your license file, and run the code to make sure the activation works.

## Mosek

* If you don't have one, obtain a Mosek license. If you have academic status, then you can obtain a [free personal academic license](https://www.mosek.com/products/academic-licenses/).
* This cell will prompt you to upload the `mosek.lic` file from your local machine.

**Please be careful to avoid publicly sharing your license file!**

We upload the file to `/tmp/mosek.lic` in order to help prevent accidental sharing of your license file.  It will be cleared at the end of each session.

Note: If you share an active session with a collaborator with your license uploaded, they could potentially copy the file out of `/tmp` and obtain your license.  

In [ ]:
import os
import os.path

import ipywidgets as widgets  # Our use requires ipywidgets >= 7.5.0
from IPython.display import display

if "MOSEKLM_LICENSE_FILE" not in os.environ:
    # If a mosek.lic file has already been uploaded, then simply use it here.
    if os.path.exists('/tmp/mosek.lic'):
        os.environ["MOSEKLM_LICENSE_FILE"] = "/tmp/mosek.lic"
    else:
        uploader = widgets.FileUpload(accept='.lic', multiple=False)
        display(uploader)

In [ ]:
if "MOSEKLM_LICENSE_FILE" not in os.environ:
    assert len(uploader.value.values()) > 0, "Please upload a license file using the Upload widget above."
    with open('/tmp/mosek.lic', 'wb') as output_file:
        output_file.write(list(uploader.value.values())[-1]['content']) 
    os.environ["MOSEKLM_LICENSE_FILE"] = "/tmp/mosek.lic"

We can confirm that Drake believes Mosek should now be available:

In [ ]:
from pydrake.solvers import MosekSolver

print(MosekSolver().enabled())

Now let's solve a trivial semidefinite program. 

$$\begin{aligned} \min_S \quad & \text{Trace}(S) \\ \text{subject to} \quad & S_{1,0} = 1, \\ & S \succeq 0. \end{aligned}$$

The known optimal solution is $ S = \begin{bmatrix} 1 & 1 \\ 1 & 1\end{bmatrix}.$

In [ ]:
from pydrake.solvers import MathematicalProgram, MosekSolver

prog = MathematicalProgram()
S = prog.NewSymmetricContinuousVariables(2, "S")

# S ≽ 0.
prog.AddPositiveSemidefiniteConstraint(S)

# S(1, 0) = 1
prog.AddBoundingBoxConstraint(1, 1, S[1, 0])

# min Trace(S)
prog.AddLinearCost(S[0,0] + S[1,1])

mosek = MosekSolver()
result = mosek.Solve(prog)
assert result.is_success()

print(result.GetSolution(S))

## Gurobi

Gurobi is not yet available in the binary releases of Drake.  We hope to enable it in the future: https://github.com/RobotLocomotion/drake/issues/10804

## Clean up

For good measure, let's delete any temporary license files now.

In [ ]:
if os.path.exists('/tmp/mosek.lic'):
    os.remove('/tmp/mosek.lic')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2b4fc509-aef2-417d-a40d-6071dfed9199' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>